# Sentiment Analysis

In [1]:
from googletrans import Translator

translator = Translator()

async def translate_text(original_text: str):
    translation = await translator.translate(original_text, dest='en')
    translated_text, original_language = translation.text, translation.src
    return translated_text, original_language

# Run it
original_text = "این غذا خیلی شوره!"

my_translation = await translate_text("این غذا خیلی شوره!")
print(my_translation)


('This food is very salty!', 'fa')


In [2]:
malay_translation = await translate_text("mak kau hijau lah")
print(malay_translation)

('Your mother is green', 'ms')


## Instantiate model, auxullary functions and first test

```bash
pip uninstall -y transformers tokenizers sentencepiece
pip install transformers==4.45.1 tokenizers==0.20.3 sentencepiece
```

In [3]:
## Sanity check
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

print("✅ Model and tokenizer loaded successfully!")

✅ Model and tokenizer loaded successfully!


In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig


MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
ROBERTA_SUPPORTED_LANGUAGES = ('ar', 'en', 'fr', 'de', 'hi', 'it', 'es', 'pt')

model = AutoModelForSequenceClassification.from_pretrained(MODEL)
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL, use_fast=False)

#/ save the model locally
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

ROBERTA_SUPPORTED_LANGUAGES = ('ar', 'en', 'fr', 'de', 'hi', 'it', 'es', 'pt')


# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

def predict_sentiment(text: str) -> str:
    processed_text = preprocess(text)
    encoded_input = tokenizer(processed_text, return_tensors='pt')
    output = model(**encoded_input)
    index_of_sentiment = output.logits.argmax().item()
    sentiment = config.id2label[index_of_sentiment]
    return sentiment

# 🧩 Sentiment prediction
import torch 
def predict_sentiment_2(text: str) -> str:
    processed_text = preprocess(text)
    # 🛡️ Tokenize safely
    encoded_input = tokenizer(
        processed_text,
        return_tensors='pt',
        truncation=True,         # <- prevent long text error
        max_length=512,          # <- enforce model limit
        padding=True
    )
    with torch.no_grad():
        output = model(**encoded_input)

    index_of_sentiment = output.logits.argmax().item()
    sentiment = config.id2label[index_of_sentiment]
    return sentiment

## First test

In [5]:
text1 = "la pizza da @michele è veramente buona https://www.youtube.com"
text2 = "این غذا خیلی شوره!"
text3 = "یه جلسه دیگه که میتونست یه ایمیل باشه 🥲"
text4 = 'Yet another one bites the dust'


translation, source_language = await translate_text(text4)
print(predict_sentiment(translation))

negative


In [6]:
# Reading the file - unpickling from RDMS_1
import pickle
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1000)
with open('./Data/COLLATE/COMMENT.pkl', 'rb') as fp:
    COLLATE_COMMENT = pickle.load(fp)
## Preview
print('COMMENT')
display(pd.concat([ COLLATE_COMMENT.head(2),  COLLATE_COMMENT.tail(2) ]))
display(COLLATE_COMMENT.dtypes)

COMMENT


,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???! | Ale Duke 2.0 Living in America - James Brown 1985 | - YouTube,0,NaN
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke hp baru gimna ya,0,NaN
15446,4tA8wtLaui4,@JokeRQuas,7 years ago,.,0,@uob
15447,gA0hQtE6TL4,@floristdahlins8520,9 years ago,",",0,@uob


Youtube_Video_IDs    object
poster_OP            object
comment_age          object
full_comment         object
like_count            int64
Youtube_Handle       object
dtype: object

In [7]:
COLLATE_COMMENT[COLLATE_COMMENT.Youtube_Handle.isna()].iloc[:5, :]

,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???! | Ale Duke 2.0 Living in America - James Brown 1985 | - YouTube,0,NaN
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke hp baru gimna ya,0,NaN
2,Y6GQJUyjDO4,@VikkiNesspara-f2u,1 month ago,happy 60th birthday Singapore 🇸🇬🇸🇬🇸🇬️🇸🇬️️🇸🇬️🇸🇬️🇸🇬️️🇸🇬️️🇸🇬️🇸🇬️ 🎇 🎇 🦁 🎆 🎆 🦁 ❤ ❤ 🕊 🕊 ✌🏾 ✌🏾 👉🏾 ❤ ❤ 🌏 👈🏾 ❤ ❤ 🕊 🕊 ✌🏾 ✌🏾 🙋🏾‍♀,0,NaN
3,GGbcl2AcSBU,@RisNandar-c9h,2 months ago,OCBC ❤ ❤,0,NaN
4,meamip7zfzI,@BozuBoyYT,11 days ago,The person doing the voice over is so good!,2,NaN


In [8]:
COLLATE_COMMENT.fillna({'Youtube_Handle' : '@channelocbc'}, inplace=True)

## Translate text and detect languages

In [9]:
import pandas as pd
import asyncio
from googletrans import Translator

translator = Translator()

async def translate_text(original_text: str):
    translation = await translator.translate(original_text, dest='en')
    return translation.text, translation.src

async def translate_series(series):
    tasks = [translate_text(text) for text in series]
    results = await asyncio.gather(*tasks)
    return results

# Run all translations asynchronously
results = await translate_series(COLLATE_COMMENT['full_comment'])
COLLATE_COMMENT[['translated_text', 'detected_lang']] = pd.DataFrame(results, columns=['translated_text', 'detected_lang'])

In [10]:
# View non-english comments
COLLATE_COMMENT[COLLATE_COMMENT['detected_lang'] != 'en']

,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle,translated_text,detected_lang
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke hp baru gimna ya,0,@channelocbc,"Sis, how do I move OCBC Mobile from an old cellphone to a new cellphone?",id
21,eO3mGG3x3E8,@AgnesWong-0924,5 months ago,好正向，阿V嘅表演真係好有吸引力 繼續enjoy人生 ❤ 😊,0,@channelocbc,"It's awesome, Ah V's performance is really attractive to continue to enjoy life ❤😊",yue
23,eO3mGG3x3E8,@TommyChu-p7o,5 months ago,原來阿V過去有咁嘅經歷,0,@channelocbc,It turns out that Ah V has had such an experience in the past,yue
24,9C_UyZJQ0Vg,@CharlieWuGunpla,5 months ago,香港電影業靠你啦！你得嘅！,0,@channelocbc,The Hong Kong film industry depends on you! You can!,yue
25,TF6ebsFMY_M,@pohchoopeh4225,5 months ago,所以上网付款最危险 😮,0,@channelocbc,So online payment is the most dangerous 😮,zh-CN
...,...,...,...,...,...,...,...,...
15392,AkxPA5mFjeI,@dctam9725,5 years ago,Hồ sơ gồm những gì ah? Lãi suất thế nào?,0,@uob,What does the profile include? What's the interest rate?,vi
15393,AkxPA5mFjeI,@HoangTranCoffee,6 years ago,"Shop mở kho hàng tại HCM, nhưng xưởng sản xuất tại Kon Tum, Hộ Kinh Doanh đăng ký tại Kon Tum, thì có được vay không ạ?",0,@uob,"The shop opens a warehouse in HCM, but the factory is in Kon Tum, and the Business House is registered in Kon Tum, can I get a loan?",vi
15394,AkxPA5mFjeI,@hoanglengoc130,5 years ago,Chỉ áp dụng cho tphcm ạ??,0,@uob,Only applies to Ho Chi Minh City??,vi
15406,4rMqRXRBGzQ&pp=0gcJCQYKAYcqIYzv,@sainikhan6530,5 years ago,Susah,0,@uob,Difficult,ms


## Infer sentiment

In [11]:
import time
start = time.time()
sentiment_results = COLLATE_COMMENT['translated_text'].apply(predict_sentiment_2)
COLLATE_COMMENT['sentiment'] = pd.Series(sentiment_results)
print("⏱️ Elapsed:", time.time() - start, "seconds")

⏱️ Elapsed: 800.3586339950562 seconds


In [12]:
# Write into COLLATE folder
with open('./Data/COLLATE/COMMENT_sentiment.pkl', 'wb') as f:
    pickle.dump(COLLATE_COMMENT, f)

# RDMS_2
> Recreate a new databse in `"./Data/RDBMS/malaysian_youtube_banks_sentiment_deduplicated.db"`

In [37]:
## Read back data
# Takes a while to read the pickle
import pickle
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 1000)
# Reading the file - unpickling
with open('./Data/abm_banks_malaysia.pkl', 'rb') as fp:
    abm_banks_df = pickle.load(fp)
## Preview

display(pd.concat([ abm_banks_df.head(1),  abm_banks_df.tail(1) ]))
print('ABM_table')
display(pd.DataFrame(abm_banks_df.dtypes, columns=['datatype']))

,Bank_Names,Hashtag_Name,Top_Person,Top_Title,Bank_Website,Youtube_Handle
0,Affin Bank Berhad,#affin_bank_berhad,Datuk Wan Razly Abdullah Wan Ali,Group CEO,https://www.affingroup.com/affin-bank-berhad,@AFFIN
25,United Overseas Bank (Malaysia) Bhd,#united_overseas_bank_(malaysia)_bhd,Ng Wei Wei,CEO,http://www.uob.com.my/,@uob


ABM_table


,datatype
Bank_Names,object
Hashtag_Name,object
Top_Person,object
Top_Title,object
Bank_Website,object
Youtube_Handle,object


In [34]:
# Reading the file - unpickling
with open('./Data/overview_youtube_2.pkl', 'rb') as fp:
    overview_df = pickle.load(fp) # New and improved 
## Preview

display(pd.concat([ overview_df.head(1),  overview_df.tail(1) ]))
print('Overview_table')
display(pd.DataFrame(overview_df.dtypes, columns=['datatype']))
display(overview_df.shape)

,ID,mail,Scraped_DateTime,phone,language,privacy_public,info_outline,person_radar,my_videos,Videos_Not_Shorts,trending_up
0,@AFFIN,,2025-10-23 19:19:33.602173,,www.youtube.com/@AFFIN,Malaysia,Joined 18 Jun 2014,4.9K subscribers,325 videos,221,"15,050,294 views"
23,@uob,,2025-10-23 19:26:02.841470,,www.youtube.com/@uob,Singapore,Joined 24 Nov 2012,29.2K subscribers,"1,286 videos",1210,"86,088,593 views"


Overview_table


,datatype
ID,object
mail,object
Scraped_DateTime,datetime64[ns]
phone,object
language,object
privacy_public,object
info_outline,object
person_radar,object
my_videos,object
Videos_Not_Shorts,int64


(24, 11)

In [35]:
# Reading the file - unpickling
with open('./Data/COLLATE/MM.pkl', 'rb') as fp:
    COLLATE_MM = pickle.load(fp)
## Preview
display(pd.concat([ COLLATE_MM.iloc[[150],:],  COLLATE_MM.tail(1) ]))
print('MM_table')
display(pd.DataFrame(COLLATE_MM.dtypes, columns=['datatype']))
display(COLLATE_MM.shape)


,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented
150,83LznfQ1aHI,@channelocbc,0:52,262.0,2024-08-29,A fresh twist on your favourites,5.0,"We have given your favourite features a makeover. Here is a smoother, more intuitive OCBC app experience.\n#ocbcbank\n\n#ocbcapp",2025-11-04 15:03:07,True
15908,tew6Ly8NDcc,@MUFGBankChannel,4:50,3859.0,2019-08-12,動画で学ぶ資産運用「つみたて投資によろしく」,0.0,No description has been added to this video,2025-11-03 16:49:01,None


MM_table


,datatype
Youtube_Video_IDs,object
Youtube_Handle,object
video_length,object
views,float64
date,datetime64[ns]
title,object
likes,float64
description,object
MM_timestamp,datetime64[ns]
is_commented,object


(15909, 10)

In [38]:
# Reading the file - unpickling - what have just done in Sentiment Analysis above.
with open('./Data/COLLATE/COMMENT_sentiment.pkl', 'rb') as fp:
    COLLATE_COMMENT = pickle.load(fp)
## Preview

display(pd.concat([ COLLATE_COMMENT.head(2),  COLLATE_COMMENT.iloc[[15000],:]]))
print('COMMENT_table')
display(pd.DataFrame(COLLATE_COMMENT.dtypes, columns=['datatype']))
display(COLLATE_COMMENT.shape)

,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle,translated_text,detected_lang,sentiment
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???! | Ale Duke 2.0 Living in America - James Brown 1985 | - YouTube,0,@channelocbc,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???! | Ale Duke 2.0 Living in America - James Brown 1985 | - YouTube,en,neutral
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke hp baru gimna ya,0,@channelocbc,"Sis, how do I move OCBC Mobile from an old cellphone to a new cellphone?",id,neutral
15000,oWyumLYSe6Q,@LisaWalker-f8b,1 year ago,Geoffrey Light,0,@uob,Geoffrey Light,en,neutral


COMMENT_table


,datatype
Youtube_Video_IDs,object
poster_OP,object
comment_age,object
full_comment,object
like_count,int64
Youtube_Handle,object
translated_text,object
detected_lang,object
sentiment,object


(15448, 9)

In [17]:
# Ensure all unique
print(len(COLLATE_MM['Youtube_Video_IDs']) == COLLATE_MM['Youtube_Video_IDs'].nunique())
# Show any duplicates - duplicates might showed up for Private Videos
bool_series = COLLATE_MM.duplicated(subset=['Youtube_Video_IDs'], keep=False)
display(COLLATE_MM[bool_series])
# Remove the dulicates, keep first instance
COLLATE_MM.drop_duplicates(subset=['Youtube_Video_IDs'], keep='first', inplace=True)

True


,Youtube_Video_IDs,Youtube_Handle,video_length,views,date,title,likes,description,MM_timestamp,is_commented


In [18]:
# Ensure all unique
print(len(COLLATE_COMMENT['Youtube_Video_IDs']) == len(set(COLLATE_COMMENT['Youtube_Video_IDs'])))
# Show any duplicates - duplicates might showed up the key below
duplicate_combo_key = ['Youtube_Video_IDs', 'poster_OP','full_comment', 'like_count']
bool_series = COLLATE_COMMENT.duplicated(subset=duplicate_combo_key, keep=False)
display(COLLATE_COMMENT[bool_series])
# Remove the dulicates, keep first instance
COLLATE_COMMENT.drop_duplicates(subset=duplicate_combo_key, keep='first', inplace=True)

## Write de-duplicated
# Write into COLLATE folder
with open('./Data/COLLATE/COMMENT_sentiment_deduplicated.pkl', 'wb') as f:
    pickle.dump(COLLATE_COMMENT, f)

False


,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle,translated_text,detected_lang,sentiment
135,HJ5YwHq3O_c&pp=0gcJCQYKAYcqIYzv,@Zam-ms7xs,5 years ago,Reset paswoand bsn,0,@channelocbc,Reset paswoand bsn,en,negative
136,HJ5YwHq3O_c&pp=0gcJCQYKAYcqIYzv,@Zam-ms7xs,5 years ago,Reset paswoand bsn,0,@channelocbc,Reset paswoand bsn,en,negative
511,sMEKjQbwUBc,@patriciavanderpoorten4102,6 months ago,Profi rechercher,0,@labanquedunmondequichange,Professional researcher,lb,neutral
512,sMEKjQbwUBc,@patriciavanderpoorten4102,6 months ago,Profi rechercher,0,@labanquedunmondequichange,Professional researcher,lb,neutral
594,vhCwaV4LMZU,@NadineBerton,8 months ago,je n'arrive pas à vous joindre,0,@labanquedunmondequichange,I can't reach you,fr,negative
595,vhCwaV4LMZU,@NadineBerton,8 months ago,je n'arrive pas à vous joindre,0,@labanquedunmondequichange,I can't reach you,fr,negative
1183,Fkw-x7bOCdE,@AdrienneGrellet,10 months ago,"Je ne retrouve pas mon n° client. Pouvez-vous me le transmettre par courrier, ainsi que mon code secret pour accéder à mes comptes . Merci",0,@labanquedunmondequichange,"I can't find my customer number. Can you send it to me by mail, along with my secret code to access my accounts. THANKS",fr,negative
1184,Fkw-x7bOCdE,@AdrienneGrellet,10 months ago,"Je ne retrouve pas mon n° client. Pouvez-vous me le transmettre par courrier, ainsi que mon code secret pour accéder à mes comptes . Merci",0,@labanquedunmondequichange,"I can't find my customer number. Can you send it to me by mail, along with my secret code to access my accounts. THANKS",fr,negative
1207,rtlVd9UQl88,@benedicteho4584,1 year ago,Débloquer ma carte,0,@labanquedunmondequichange,Unblock my card,fr,negative
1214,rtlVd9UQl88,@anniedel9988,4 years ago,Assistant digital,0,@labanquedunmondequichange,Assistant digital,en,neutral


## Final form of COLLATE_COMMENT
> Now COLLATE_COMMENT is in the best form, we can create the new database.

In [15]:
# Reading the file - unpickling
import pickle
import pandas as pd
with open('./Data/COLLATE/COMMENT_sentiment_deduplicated.pkl', 'rb') as fp:
    COLLATE_COMMENT = pickle.load(fp)
## Preview
print('COMMENT_Deduplicated')
display(pd.concat([ COLLATE_COMMENT.head(2),  COLLATE_COMMENT.tail(2) ]))
display(COLLATE_COMMENT.dtypes)
display(COLLATE_COMMENT.shape)

COMMENT_Deduplicated


,Youtube_Video_IDs,poster_OP,comment_age,full_comment,like_count,Youtube_Handle,translated_text,detected_lang,sentiment
0,FmQz15WJkWE,@tinkerlee-r9z,3 weeks ago,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???!...,0,@channelocbc,BECAREFUL WHAT YOU WISH FOR - CIRCUS ???!...,en,neutral
1,zb3oKcLv1yA,@kayanemahberkah,1 month ago,kak cara pindahin ocbc mobile dari hp lama ke ...,0,@channelocbc,"Sis, how do I move OCBC Mobile from an old cel...",id,neutral
15446,4tA8wtLaui4,@JokeRQuas,7 years ago,.,0,@uob,.,en,neutral
15447,gA0hQtE6TL4,@floristdahlins8520,9 years ago,",",0,@uob,",",en,neutral


Youtube_Video_IDs    object
poster_OP            object
comment_age          object
full_comment         object
like_count            int64
Youtube_Handle       object
translated_text      object
detected_lang        object
sentiment            object
dtype: object

(15363, 9)

In [20]:
import sqlite3
import pandas as pd

# 🧩 assume you already have these DataFrames defined:
# ABM_table, Overview_table, MM_df, COMMENT_df

# ----------------------------------------------------
# 1️⃣ Connect to database (creates youtube_data.db if not exists)
# ----------------------------------------------------
conn = sqlite3.connect("./Data/RDBMS/malaysian_youtube_banks_sentiment_deduplicated.db")
cursor = conn.cursor()

# Always enable foreign key enforcement in SQLite
cursor.execute("PRAGMA foreign_keys = ON;")

# ----------------------------------------------------
# 2️⃣ Define schema (drop old tables and recreate)
# ----------------------------------------------------
schema_script = """
PRAGMA foreign_keys = ON;

DROP TABLE IF EXISTS COMMENT_table;
DROP TABLE IF EXISTS MM_table;
DROP TABLE IF EXISTS Overview_table;
DROP TABLE IF EXISTS ABM_table;

CREATE TABLE ABM_table (
    Bank_Names TEXT,
    Hashtag_Name TEXT,
    Top_Person TEXT,
    Top_Title TEXT,
    Bank_Website TEXT,
    Youtube_Handle TEXT PRIMARY KEY
);

CREATE TABLE Overview_table (
    ID TEXT PRIMARY KEY,
    mail TEXT,
    Scraped_DateTime DATETIME,
    phone TEXT,
    language TEXT,
    privacy_public TEXT,
    info_outline TEXT,
    person_radar TEXT,
    my_videos TEXT,
    Videos_Not_Shorts INTEGER,
    trending_up TEXT,
    FOREIGN KEY (ID) REFERENCES ABM_table (Youtube_Handle)
);

CREATE TABLE MM_table (
    Youtube_Video_IDs TEXT PRIMARY KEY,
    Youtube_Handle TEXT NOT NULL,
    video_length INTEGER,   -- should already be converted to seconds
    views REAL,
    date DATETIME,
    title TEXT,
    likes REAL,
    description TEXT,
    MM_timestamp DATETIME,
    is_commented TEXT,
    FOREIGN KEY (Youtube_Handle) REFERENCES ABM_table (Youtube_Handle)
);

CREATE TABLE COMMENT_table (
    Youtube_Video_IDs TEXT NOT NULL,
    Youtube_Handle TEXT,
    poster_OP TEXT,
    comment_age TEXT,
    full_comment TEXT,
    like_count INTEGER,
    translated_text TEXT,
    detected_lang TEXT,
    sentiment TEXT,
    FOREIGN KEY (Youtube_Video_IDs) REFERENCES MM_table (Youtube_Video_IDs)
);
"""

cursor.executescript(schema_script)
print("✅ Tables created successfully.")

# ----------------------------------------------------
# 3️⃣ Insert data into tables (in dependency order)
# ----------------------------------------------------
# Each to_sql() appends data to the pre-defined schema.
# index=False prevents adding an unnecessary index column.

abm_banks_df.to_sql("ABM_table", conn, if_exists="append", index=False)
overview_df.to_sql("Overview_table", conn, if_exists="append", index=False)
COLLATE_MM.to_sql("MM_table", conn, if_exists="append", index=False)
COLLATE_COMMENT.to_sql("COMMENT_table", conn, if_exists="append", index=False)

print("✅ DataFrames inserted successfully.")

# ----------------------------------------------------
# 4️⃣ Commit changes and close connection
# ----------------------------------------------------
conn.commit()
conn.close()

print("💾 Database committed and connection closed.")

✅ Tables created successfully.
✅ DataFrames inserted successfully.
💾 Database committed and connection closed.


# SQL-to-Text Agent

In [21]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFacePipeline

In [22]:
from dotenv import load_dotenv
load_dotenv()

True

In [23]:
db = SQLDatabase.from_uri("sqlite:///./Data/RDBMS/malaysian_youtube_banks.db", sample_rows_in_table_info=0)

def get_schema(_):
    return db.get_table_info()


def run_query(query):
    print(f'Query being run: {query} \n\n')
    return db.run(query)

In [24]:
print(get_schema(''))


CREATE TABLE "ABM_table" (
	"Bank_Names" TEXT, 
	"Hashtag_Name" TEXT, 
	"Top_Person" TEXT, 
	"Top_Title" TEXT, 
	"Bank_Website" TEXT, 
	"Youtube_Handle" TEXT, 
	PRIMARY KEY ("Youtube_Handle")
)


CREATE TABLE "COMMENT_table" (
	"Youtube_Video_IDs" TEXT NOT NULL, 
	"Youtube_Handle" TEXT, 
	"poster_OP" TEXT, 
	comment_age TEXT, 
	full_comment TEXT, 
	like_count INTEGER, 
	FOREIGN KEY("Youtube_Video_IDs") REFERENCES "MM_table" ("Youtube_Video_IDs")
)


CREATE TABLE "MM_table" (
	"Youtube_Video_IDs" TEXT, 
	"Youtube_Handle" TEXT NOT NULL, 
	video_length INTEGER, 
	views REAL, 
	date DATETIME, 
	title TEXT, 
	likes REAL, 
	description TEXT, 
	"MM_timestamp" DATETIME, 
	is_commented TEXT, 
	PRIMARY KEY ("Youtube_Video_IDs"), 
	FOREIGN KEY("Youtube_Handle") REFERENCES "ABM_table" ("Youtube_Handle")
)


CREATE TABLE "Overview_table" (
	"ID" TEXT, 
	mail TEXT, 
	"Scraped_DateTime" DATETIME, 
	phone TEXT, 
	language TEXT, 
	privacy_public TEXT, 
	info_outline TEXT, 
	person_radar TEXT, 
	my_vid

In [25]:
def get_llm(load_from_hugging_face=False):
    if load_from_hugging_face:
        api_key = os.environ.get("OPENROUTER_API_KEY")
        if not api_key:
            raise ValueError("Missing HYPERBOLIC_API_KEY environment variable.")
            print(f"✅ Loaded API key: {api_key[:5]}... (length: {len(api_key)})")
        llm = HuggingFaceEndpoint(
            repo_id="Qwen/Qwen2.5-VL-7B-Instruct",
            task="text-generation",
            provider="hyperbolic",  # set your provider here
            api_key=api_key,  # ✅ add this line
        )

        return ChatHuggingFace(llm=llm)
    
    return ChatOpenAI(model="gpt-4o", temperature=0.0)


def write_sql_query(llm):
    template = """Based on the table schema below, write a SQL query that would answer the user's question:
    {schema}

    Question: {question}
    SQL Query:"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Given an input question, convert it to a SQL query. No pre-amble. "
            "Please do not return anything else apart from the SQL query, no prefix aur suffix quotes, no sql keyword, nothing please"),
            ("human", template),
        ]
    )

    return (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | llm
        | StrOutputParser()
    )

In [26]:
def answer_user_query(query, llm):
    template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
    {schema}

    Question: {question}
    SQL Query: {query}
    SQL Response: {response}"""

    prompt_response = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
            ),
            ("human", template),
        ]
    )

    full_chain = (
        RunnablePassthrough.assign(query=write_sql_query(llm))
        | RunnablePassthrough.assign(
            schema=get_schema,
            response=lambda x: run_query(x["query"]),
        )
        | prompt_response
        | llm
    )

    return full_chain.invoke({"question": query})

In [27]:
# write_sql_query(llm=get_llm(load_from_hugging_face=True)).invoke({"question": "Give me 10 Artists"})
query = 'Give some Youtube_Video IDs from AmBank youtube channel. Refer'
response = answer_user_query(query, llm=get_llm(load_from_hugging_face=False))
print(response.content)

Query being run: SELECT MM_table.Youtube_Video_IDs 
FROM MM_table 
JOIN ABM_table ON MM_table.Youtube_Handle = ABM_table.Youtube_Handle 
WHERE ABM_table.Bank_Names = 'AmBank' 


There are no YouTube video IDs available from the AmBank YouTube channel in the database.


### Check if api key is active

In [28]:
import requests, os
from dotenv import load_dotenv
key = os.environ.get("HYPERBOLIC_API_KEY")
r = requests.get("https://api.hyperbolic.xyz/v1/models", headers={"Authorization": f"Bearer {key}"})
print(r.status_code, r.text)

200 {"object":"list","data":[{"id":"Qwen/Qwen2.5-72B-Instruct","created":1762682100,"object":"model","owned_by":"Hyperbolic","number_of_inference_nodes":null,"supports_chat":true,"supports_image_input":false,"supports_tools":false,"context_length":131072,"input_price":0.4,"output_price":0.4},{"id":"meta-llama/Meta-Llama-3-70B-Instruct","created":1762682100,"object":"model","owned_by":"Hyperbolic","number_of_inference_nodes":null,"supports_chat":true,"supports_image_input":false,"supports_tools":false,"context_length":8192,"input_price":0.4,"output_price":0.4},{"id":"deepseek-ai/DeepSeek-V3","created":1762682100,"object":"model","owned_by":"Hyperbolic","number_of_inference_nodes":null,"supports_chat":true,"supports_image_input":false,"supports_tools":false,"context_length":131072,"input_price":0.25,"output_price":0.25},{"id":"deepseek-ai/DeepSeek-V3-0324","created":1762682100,"object":"model","owned_by":"Hyperbolic","number_of_inference_nodes":null,"supports_chat":true,"supports_image_i

## Putting it all together

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint, HuggingFacePipeline

from dotenv import load_dotenv
load_dotenv()

db = SQLDatabase.from_uri("sqlite:///./Data/RDBMS/malaysian_youtube_banks.db", sample_rows_in_table_info=0)

def get_schema(_):
    return db.get_table_info()


def run_query(query):
    print(f'Query being run: {query} \n\n')
    return db.run(query)

print(get_schema(''))


def get_llm(load_from_hugging_face=False):
    if load_from_hugging_face:
        api_key = os.environ.get("OPENROUTER_API_KEY")
        if not api_key:
            raise ValueError("Missing HYPERBOLIC_API_KEY environment variable.")
            print(f"✅ Loaded API key: {api_key[:5]}... (length: {len(api_key)})")
        llm = HuggingFaceEndpoint(
            repo_id="Qwen/Qwen2.5-VL-7B-Instruct",
            task="text-generation",
            provider="hyperbolic",  # set your provider here
            api_key=api_key,  # ✅ add this line
        )

        return ChatHuggingFace(llm=llm)
    
    return ChatOpenAI(model="gpt-4o", temperature=0.0)


def write_sql_query(llm):
    template = """Based on the table schema below, write a SQL query that would answer the user's question:
    {schema}

    Question: {question}
    SQL Query:"""

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "Given an input question, convert it to a SQL query. No pre-amble. "
            "Please do not return anything else apart from the SQL query, no prefix aur suffix quotes, no sql keyword, nothing please"),
            ("human", template),
        ]
    )

    return (
        RunnablePassthrough.assign(schema=get_schema)
        | prompt
        | llm
        | StrOutputParser()
    )


def answer_user_query(query, llm):
    template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
    {schema}

    Question: {question}
    SQL Query: {query}
    SQL Response: {response}"""

    prompt_response = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "Given an input question and SQL response, convert it to a natural language answer. No pre-amble.",
            ),
            ("human", template),
        ]
    )

    full_chain = (
        RunnablePassthrough.assign(query=write_sql_query(llm))
        | RunnablePassthrough.assign(
            schema=get_schema,
            response=lambda x: run_query(x["query"]),
        )
        | prompt_response
        | llm
    )

    return full_chain.invoke({"question": query})

# import os
# os.environ



CREATE TABLE "ABM_table" (
	"Bank_Names" TEXT, 
	"Hashtag_Name" TEXT, 
	"Top_Person" TEXT, 
	"Top_Title" TEXT, 
	"Bank_Website" TEXT, 
	"Youtube_Handle" TEXT, 
	PRIMARY KEY ("Youtube_Handle")
)


CREATE TABLE "COMMENT_table" (
	"Youtube_Video_IDs" TEXT NOT NULL, 
	"Youtube_Handle" TEXT, 
	"poster_OP" TEXT, 
	comment_age TEXT, 
	full_comment TEXT, 
	like_count INTEGER, 
	FOREIGN KEY("Youtube_Video_IDs") REFERENCES "MM_table" ("Youtube_Video_IDs")
)


CREATE TABLE "MM_table" (
	"Youtube_Video_IDs" TEXT, 
	"Youtube_Handle" TEXT NOT NULL, 
	video_length INTEGER, 
	views REAL, 
	date DATETIME, 
	title TEXT, 
	likes REAL, 
	description TEXT, 
	"MM_timestamp" DATETIME, 
	is_commented TEXT, 
	PRIMARY KEY ("Youtube_Video_IDs"), 
	FOREIGN KEY("Youtube_Handle") REFERENCES "ABM_table" ("Youtube_Handle")
)


CREATE TABLE "Overview_table" (
	"ID" TEXT, 
	mail TEXT, 
	"Scraped_DateTime" DATETIME, 
	phone TEXT, 
	language TEXT, 
	privacy_public TEXT, 
	info_outline TEXT, 
	person_radar TEXT, 
	my_vid

## Invoke

In [30]:
# write_sql_query(llm=get_llm(load_from_hugging_face=True)).invoke({"question": "Give me 10 Artists"})
query = 'Give some Youtube IDs from @AmBankTV youtube channel'
# query = 'What is the most liked Youtube_Handle? What is the like count?'
response = answer_user_query(query, llm=get_llm(load_from_hugging_face=False))
print(response.content)

Query being run: SELECT "Youtube_Video_IDs" FROM "MM_table" WHERE "Youtube_Handle" = '@AmBankTV' 


Here are some YouTube video IDs from the @AmBankTV YouTube channel: p7bPO1SfHJc, xceiS_r8WlQ, s9pNJYzRvxs, FPGhDYBIlVw, obl5r-aOqy4, sh0XVQDBBsY, -EmdeyJqKs0, lyUjdro80JU, Og6vcBivg9s, iWlD8iSavE8, Gxqo6ATyNRk, dakhg2Y5dmA, NuvL9E1tj78, DQoAgLVNmQQ, m6vp8iRZQho, v0fGb5HYdj8, 1CuoWYO4g3A, 8R-PJGUtckQ, kUaME0yWLlk, Ingpxi2P9l4, NHjDiHutLI8, QS-qG6qjK0c, pgOf1fNtBFM, DneKHHrKTjg, Ucridpq3AE8, n0BI-kBRPE8, GXsRutMMGyk, 9mOg79iCBvo, TIaFNvG9d_s, xuVsf7ViE80, zCHrjR2eRt0&pp=0gcJCQYKAYcqIYzv, ngG-L1P8yB8, 6PK-YB5cd6I, 6buWnO2MHR0, O7lsF5QIUoE, pQkm_E0MbC4, 4NIUwMTwHKQ, dvXU7IcXiaQ, ySvLzc903NI, vVVF4u4aBF8, y86BUfJJ6F8, nB_SGdsekEs, ATZXj7DNUM0&pp=0gcJCQYKAYcqIYzv, zn_1LtSBb9M, VYLfZXt6eI8, bQTveY0KRwk, ckXP_hL6ONs, 7dJcjmjhW0k, tfPe0ZGIDQE, BuppOchl91w, vL0E86l40pE, LiC4rIiZwLM, pjWz2AVxcQs, 2wsiMY-Se6c, cnEzC8kAPBo, -Owb2J300s4, kG7LUO_LQQ0, hR5EbQ7TvIQ, d-TnDKkWYLk, uQKdMhvKUmU, JV3s5I2kadY,

## Appendix

In [ ]:
## Postive test cases
query1 = "What is the most viewed video in MM_table? What is the video title and video_ID?"
query2 = 'Are there any repeated poster_OPs across Youtube_Handles within the COMMENT_table?'


In [9]:
import re
string  = """
SELECT MM_table.title, COUNT(COMMENT_table.sentiment) AS negative_comment_count, MM_table.likes
FROM MM_table
JOIN COMMENT_table ON MM_table.Youtube_Video_IDs = COMMENT_table.Youtube_Video_IDs
WHE;"""

sql_match = re.search(r"SELECT.*?;", string, re.IGNORECASE | re.DOTALL | re.MULTILINE)

sql_match.group(0).strip()

'SELECT MM_table.title, COUNT(COMMENT_table.sentiment) AS negative_comment_count, MM_table.likes\nFROM MM_table\nJOIN COMMENT_table ON MM_table.Youtube_Video_IDs = COMMENT_table.Youtube_Video_IDs\nWHE;'

In [19]:
COLLATE_COMMENT.groupby(['Youtube_Handle', 'sentiment']).agg(col_1=pd.NamedAgg(column='sentiment', aggfunc='count')).loc['@AFFIN',:]

,col_1
sentiment,
negative,165
neutral,96
positive,319


In [20]:
COLLATE_COMMENT.columns

Index(['Youtube_Video_IDs', 'poster_OP', 'comment_age', 'full_comment',
       'like_count', 'Youtube_Handle', 'translated_text', 'detected_lang',
       'sentiment'],
      dtype='object')

In [6]:
data = [('LiFxYCLSvb4&pp=0gcJCQYKAYcqIYzv', '@channelocbc', '1:07', 90.0, '2025-10-15 00:00:00', 'OCBC Structured Notes: Customise Your Risk and Gains to Your Investment Appetite', 3.0, 'Enjoy the flexibility of attaining exposure to assets without direct ownership of it. Choose from a wide range of assets including interest rates, foreign exchange rates, equities, market indices, credit, commodity prices.', '2025-11-02 22:59:13', '0'), ('N9SBRNEAnhI', '@channelocbc', '2:11', 514.0, '2025-10-10 00:00:00', 'OCBC - Banking on Us - Episode 1', 16.0, 'While the city sleeps, our teams are wide awake.\n\nEpisode 1 of Banking on Us takes you into the world of Scheduled Maintenance, a regular but critical operation that keeps us strong, secure and ready.\n\n🎬 Watch the full story and meet the people working tirelessly behind the scenes.', '2025-11-02 22:59:33', '0'), ('PLqT8W6JNDY', '@channelocbc', '1:47', 104.0, '2025-10-03 00:00:00', 'OCBC One Connect - Integrated Ecosystem Support (China-Indonesia)', 1.0, 'OCBC One Connect - Integrated Ecosystem Support (China-Indonesia)', '2025-11-02 22:59:52', '0'), ('4UBrl92xXOU', '@channelocbc', '1:59', 79932.0, '2025-10-01 00:00:00', 'Behind the scenes of our 2025 brand campaign – Purpose is about lifting others.', 7.0, 'Watch how our latest brand campaign stories get put together, through the sharing of inspiring tales of four individuals from Singapore, Hong Kong, Indonesia, and Malaysia - Each one of them bravely pursuing their dreams and transforming the lives of countless others in the process.\n\nPurpose...', '2025-11-02 23:00:12', '0'), ('cYtlA9Q7UAo', '@channelocbc', '4:18', 178.0, '2025-09-30 00:00:00', 'OCBC華僑銀行市場熱點: 預期今年減息三次 (22-09-2025)', 2.0, 'OCBC華僑銀行財富管理專家團隊預期，美國就業數據疲弱，將促使聯儲局加快減息，預期聯儲局將於9月、10月和12月各減息25個基點，合共減息三次，2025年底聯邦基金利率將降至3.50%-3.75%，聯儲局重啟減息將為金融市場注入新動力。', '2025-11-02 23:00:32', '0')]
pd.DataFrame(data)

,0,1,2,3,4,5,6,7,8,9
0,LiFxYCLSvb4&pp=0gcJCQYKAYcqIYzv,@channelocbc,1:07,90.0,2025-10-15 00:00:00,OCBC Structured Notes: Customise Your Risk and Gains to Your Investment Appetite,3.0,"Enjoy the flexibility of attaining exposure to assets without direct ownership of it. Choose from a wide range of assets including interest rates, foreign exchange rates, equities, market indices, credit, commodity prices.",2025-11-02 22:59:13,0
1,N9SBRNEAnhI,@channelocbc,2:11,514.0,2025-10-10 00:00:00,OCBC - Banking on Us - Episode 1,16.0,"While the city sleeps, our teams are wide awake.\n\nEpisode 1 of Banking on Us takes you into the world of Scheduled Maintenance, a regular but critical operation that keeps us strong, secure and ready.\n\n🎬 Watch the full story and meet the people working tirelessly behind the scenes.",2025-11-02 22:59:33,0
2,PLqT8W6JNDY,@channelocbc,1:47,104.0,2025-10-03 00:00:00,OCBC One Connect - Integrated Ecosystem Support (China-Indonesia),1.0,OCBC One Connect - Integrated Ecosystem Support (China-Indonesia),2025-11-02 22:59:52,0
3,4UBrl92xXOU,@channelocbc,1:59,79932.0,2025-10-01 00:00:00,Behind the scenes of our 2025 brand campaign – Purpose is about lifting others.,7.0,"Watch how our latest brand campaign stories get put together, through the sharing of inspiring tales of four individuals from Singapore, Hong Kong, Indonesia, and Malaysia - Each one of them bravely pursuing their dreams and transforming the lives of countless others in the process.\n\nPurpose...",2025-11-02 23:00:12,0
4,cYtlA9Q7UAo,@channelocbc,4:18,178.0,2025-09-30 00:00:00,OCBC華僑銀行市場熱點: 預期今年減息三次 (22-09-2025),2.0,OCBC華僑銀行財富管理專家團隊預期，美國就業數據疲弱，將促使聯儲局加快減息，預期聯儲局將於9月、10月和12月各減息25個基點，合共減息三次，2025年底聯邦基金利率將降至3.50%-3.75%，聯儲局重啟減息將為金融市場注入新動力。,2025-11-02 23:00:32,0
